# Track width a wheelbase study
### Dave Yonkers, June 2020

## Aero:

In [7]:
import numpy as np
import pandas as pd

# data from James' sims
data = [[45, 208,   89,   48,  8.6,   78, 29.5,    89, 35.5, 0.33],
        [46, 233, 93.6, 50.3,    9, 94.4, 30.5,    96,   35, 0.32],
        [47, 229, 92.5, 49.8,  8.8,   93, 28.7,  95.5, 38.5, 0.30],
        [48, 257, 95.5, 56.8,  9.9,  108,   30,  99.3, 40.3, 0.36],
        [49, 268, 95.8, 59.5, 10.1,  114, 28.4,   102,   41, 0.37],
        [50, 285,  100, 65.8, 10.9,  120,   30, 108.5, 43.5, 0.37]]
aero_df = pd.DataFrame(data)
aero_df.columns = ["track_width", "downforce", "drag", "fw_downforce", "fw_drag", "tray_downforce", "tray_drag", "rw_downforce", "rw_drag", "abal"]
aero_df.set_index("track_width", inplace=True)

In [8]:
aero_df

,downforce,drag,fw_downforce,fw_drag,tray_downforce,tray_drag,rw_downforce,rw_drag,abal
track_width,,,,,,,,,
45,208,89.0,48.0,8.6,78.0,29.5,89.0,35.5,0.33
46,233,93.6,50.3,9.0,94.4,30.5,96.0,35.0,0.32
47,229,92.5,49.8,8.8,93.0,28.7,95.5,38.5,0.30
48,257,95.5,56.8,9.9,108.0,30.0,99.3,40.3,0.36
49,268,95.8,59.5,10.1,114.0,28.4,102.0,41.0,0.37
50,285,100.0,65.8,10.9,120.0,30.0,108.5,43.5,0.37


### Create a regression
only use 49 & 50 track width, since we know it should "in theory" scale linearly

In [26]:
# generate a polynomial and a function to predict y-values
coefs = np.polyfit(aero_df.index[aero_df.index >= 49], aero_df["downforce"][aero_df.index >= 49], deg=1)
downforce_function = np.poly1d(coefs)

# now for drag
coefs = np.polyfit(aero_df.index[aero_df.index >= 49], aero_df["drag"][aero_df.index >= 49], deg=1)
drag_function = np.poly1d(coefs)

aero_df["pred_downforce"] = downforce_function(aero_df.index)
aero_df["pred_drag"] = drag_function(aero_df.index)

In [33]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# create subplots objects
fig = make_subplots(rows=1, cols=2, subplot_titles=("Downforce vs Track Width", "Drag vs Track Width"))

# create the scatter traces to add
downforce_trace = go.Scattergl(x=aero_df.index, y=aero_df["downforce"], mode='markers', name="CFD downforce")
drag_trace = go.Scattergl(x=aero_df.index, y=aero_df["drag"], mode='markers', name="CFD drag")

# create the line traces to add
pred_downforce_trace = go.Scattergl(x=aero_df.index, y=aero_df["pred_downforce"], mode='lines', name="downforce regression")
pred_drag_trace = go.Scattergl(x=aero_df.index, y=aero_df["pred_drag"], mode='lines', name="drag regression")

# add the traces to the subplots
fig.add_trace(downforce_trace, row=1, col=1)
fig.add_trace(drag_trace, row=1, col=2)
fig.add_trace(pred_downforce_trace, row=1, col=1)
fig.add_trace(pred_drag_trace, row=1, col=2)

# additional formatting
fig.update_xaxes(title_text="Track Width", row=1, col=1)
fig.update_xaxes(title_text="Track Width", row=1, col=2)
fig.update_yaxes(title_text="Downforce (lbs)", row=1, col=1)
fig.update_yaxes(title_text="Drag (lbs)", row=1, col=2)

fig.show()

In [37]:
# create scale factors and a regression to generate them along the lines shown above
aero_df["downforce_scale"] = aero_df["downforce"] / aero_df["downforce"][50]
aero_df["drag_scale"]      = aero_df["drag"]      / aero_df["drag"][50]

# generate regressions for the 49 & 50 inch TWs
coefs = np.polyfit(aero_df.index[aero_df.index >= 49], aero_df["downforce_scale"][aero_df.index >= 49], deg=1)
downforce_scale_factor = np.poly1d(coefs)

# now for drag
coefs = np.polyfit(aero_df.index[aero_df.index >= 49], aero_df["drag_scale"][aero_df.index >= 49], deg=1)
drag_scale_factor = np.poly1d(coefs)

aero_df

,downforce,drag,fw_downforce,fw_drag,tray_downforce,tray_drag,rw_downforce,rw_drag,abal,pred_downforce,pred_drag,downforce_scale,drag_scale
track_width,,,,,,,,,,,,,
45,208,89.0,48.0,8.6,78.0,29.5,89.0,35.5,0.33,200.0,79.0,0.729825,0.890
46,233,93.6,50.3,9.0,94.4,30.5,96.0,35.0,0.32,217.0,83.2,0.817544,0.936
47,229,92.5,49.8,8.8,93.0,28.7,95.5,38.5,0.30,234.0,87.4,0.803509,0.925
48,257,95.5,56.8,9.9,108.0,30.0,99.3,40.3,0.36,251.0,91.6,0.901754,0.955
49,268,95.8,59.5,10.1,114.0,28.4,102.0,41.0,0.37,268.0,95.8,0.940351,0.958
50,285,100.0,65.8,10.9,120.0,30.0,108.5,43.5,0.37,285.0,100.0,1.000000,1.000


## Creating the runs array

In [53]:
min_tw = 40
max_tw = 55
step_tw = 1
tws = np.arange(min_tw, max_tw + step_tw, step_tw)

min_wb = 60.5
max_wb = 80.5
step_wb = 1
wbs = np.arange(min_wb, max_wb + step_wb, step_wb)

num_tracks = 6
# 0 = accel
# 1 = skidpad
# 2 = autocross
# 3 = slalom_min
# 4 = slalom_med
# 5 = slalom_max
# 6 = endurance - not running at the moment

runs = []

for track in range(num_tracks):
    for tw in tws:
        for wb in wbs:
            
            # calculate the aero scale factors
            downforce_scaler = downforce_scale_factor(tw)
            drag_scaler = drag_scale_factor(tw)
            
            # create the run array
            run_array = (track, tw, wb, downforce_scaler, drag_scaler)
            runs.append(run_array)
            
# turn the runs list into a dataframe
runs_df = pd.DataFrame(runs)
runs_df.columns = ("track", "tw", "wb", "df_scale", "drag_scale")
runs_df

,track,tw,wb,df_scale,drag_scale
0,0,40,60.5,0.403509,0.58
1,0,40,61.5,0.403509,0.58
2,0,40,62.5,0.403509,0.58
3,0,40,63.5,0.403509,0.58
4,0,40,64.5,0.403509,0.58
...,...,...,...,...,...
2011,5,55,76.5,1.298246,1.21
2012,5,55,77.5,1.298246,1.21
2013,5,55,78.5,1.298246,1.21
2014,5,55,79.5,1.298246,1.21


In [ ]:
# save the runs array as a csv!
runs_df.to_csv("")